In [1]:
! conda install -c conda-forge keras==2.3.1 -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ubuntu/miniconda3/envs/py3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipython-7.20.0             |   py37h888b3d9_2         1.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.1 MB

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2020.12.5-py37h06a~ --> conda-forge::certifi-2020.12.5-py37h89c1867_1
  ipython                             7.12.0-py37h5ca1d4c_0 --> 7.20.0-py37h888b3d9_2

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certif

In [5]:

# for keras 2.3.1
from tensorflow.keras.models import load_model

import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
def mergeDF(path,to_dir,file_name):
    appended_data = []
    for infile in glob.glob(path):
        #print(infile)
        data = pd.read_pickle(infile)
        # store DataFrame in list
        appended_data.append(data)
    result_path=to_dir+"/"+file_name
    print("Save:",result_path)
    appended_data = pd.concat(appended_data)
    appended_data.sort_values(by=['ID'], inplace=True)
    appended_data.to_pickle(result_path)
    return appended_data

def predict_CNN(path,to_dir,existing_result):
    appended_pkl = []
    for infile in glob.glob(path):  
     #print("Read:",infile)
        file_name = os.path.basename(infile)
        result_path=to_dir+"/"+file_name.replace("pkl", "ML.pkl")
        if result_path in existing_result :
        # print("found then skip : " , result)
            continue
        else:
            df = pd.read_pickle(infile)        
            X= np.array(df['reps'].to_list())
            X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
            y_probas = learner.predict(X_test)
            threshold = 0.5
            _y = np.where(y_probas > threshold, 1, 0)
            df.drop(columns=['reps'],inplace =True)
            df['class'] = _y
            #print("Save:",result_path)
            df.to_pickle(result_path) 

    print("Complete")

def read_exists_result(path):
    existing_result = []
    for infile in glob.glob(path+"/*.pkl"):
        # print(infile)
        existing_result.append(infile)
    print(len(existing_result))
    return existing_result

In [7]:
import os
deployed_path = "/mnt/vdb/thesis/best_CNN.1900._30.hdf5"
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
#import tensorflow as tf
#with tf.device('/cpu:0'):
learner = load_model(deployed_path)
learner.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1900, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1900, 1900)        7600      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 950, 1900)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1805000)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               231040128 
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

## DECockRoach

In [4]:
path = "/mnt/vdb/DECockroach/cd100/result_2_ShortCNN"
if not os.path.exists(path):
    os.makedirs(path)
existing_result = read_exists_result(path)

0


In [5]:
# Predict
predict_CNN("/mnt/vdb/DECockroach/cd100/reps/*.pkl",path,existing_result)

Complete


In [6]:
# write result
result_df =mergeDF(path+"/*.pkl",path,"DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/cd100/result_2_ShortCNN/DECockroach.len15.MLResult.plk


,ID,length,class
0,0,17,0
1,1,17,0
2,3,18,0
3,4,30,0
4,5,17,0
...,...,...,...
199,894298,17,0
200,894299,19,0
201,894303,17,1
202,894304,17,0


In [7]:
result_df[result_df["class"]== 1]

,ID,length,class
9,10,18,1
10,12,20,1
13,16,19,1
15,18,20,1
17,20,19,1
...,...,...,...
189,894282,32,1
190,894284,30,1
193,894288,17,1
195,894293,28,1


In [10]:
result_df[result_df["class"]== 1]

,ID,length,class
9,10,18,1
10,12,20,1
13,16,19,1
14,17,17,1
16,19,17,1
...,...,...,...
193,894288,17,1
195,894293,28,1
199,894298,17,1
201,894303,17,1


## BAT

In [8]:
path = "/mnt/vdb/Bat/cd100/result_2_ShortCNN"
if not os.path.exists(path):
    os.makedirs(path)
existing_result = read_exists_result(path)

0


In [9]:
# Predict
predict_CNN("/mnt/vdb/Bat/cd100/reps/*.pkl",path,existing_result)

Complete


In [10]:
# write result
result_df =mergeDF(path+"/*.pkl",path,"Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/cd100/result_2_ShortCNN/Bat.len10.MLResult.plk


,ID,length,class
0,9,275,1
1,19,47,1
2,32,151,1
3,54,199,0
4,61,152,1
...,...,...,...
74,12568817,51,1
75,12568833,45,1
76,12568849,117,1
77,12568850,174,1
